# Handling Missing Data

In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to respresent it, and demonstrate some built-in Pandas tools for handling missing data in Python. We'll refer to missing data in general as _null_, _NaN_, or _NA_ values.

## Trade-Offs in Missing Data Conventions

There are generally two straegies to indicate  the presence of missing data in a table or DataFrame. 

1. Using a _mask_ that globally indicates missing values:<br>
In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.
2. Choosing a _sentinel value_ that indicates a missing entry:<br>
In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

## `None`: Pythonic missing data

The first sentinel value used by Pandas is `None`, a Python singleton object that is often used for missing data in Python code.

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

This `dtype=object` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects. While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [5]:
for dtype in ['object', 'int']:
    print('dtype =', dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
63.2 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.48 ms ± 73.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



The use of Python objects in an array also means that if you perform aggregations like `sum()` or `min()` across an array with a `None` value, you will generally get an error.

In [6]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## `NaN`: Missing numerical data

The other missing data representation, `NaN` (acronym for _Not a Number_), is different; it is a special floating-point value recognized by all systems that use the standard floating-point value.

In [8]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code. You should be aware that `NaN` is a bit like a data virus–it infects any other object it touches. Regardless of the operation, the result of arithmetic with `NaN` will be another `NaN`:

In [9]:
# nan as data virus
1 + np.nan

nan

In [10]:
0 * np.nan

nan

In [11]:
vals2.sum(), vals2.min(), vals2.max()

C:\Users\Batakrishna\Anaconda3\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
C:\Users\Batakrishna\Anaconda3\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


(nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:

In [12]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

## NaN and None in Pandas

`NaN` and `None` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [13]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to `np.nan`, it will automatically be upcast to a floating-point type to accommodate the NA:

In [26]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [27]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Notice that dtype changed from int32 to float64.

## Operating on Null Values

Methods for detecting, removing, and replacing null values in Pandas data structures:

* `isnull()`: Generate a boolean mask indicating missing values
* `notnull()`: Opposite of `isnull()`
* `dropna()`: Return a filtered version of the data
* `fillna()`: Return a copy of the data with missing values filled or imputed

## Detecting null values 

Pandas data structures have two useful methods for detecting null data:`isnull()` and `notnull()`. Either one will return a Boolean mask over the data. For example:

In [28]:
data = pd.Series([1, np.nan, 'hello', None])

In [34]:
print('data.isnull()\n', data.isnull(), "\n")
print('data.isna()\n', data.isna())

data.isnull()
 0    False
1     True
2    False
3     True
dtype: bool 

data.isna()
 0    False
1     True
2    False
3     True
dtype: bool


In [38]:
# Using Boolean masking to Extract non NaN data
data[data.notnull()]

0        1
2    hello
dtype: object

## Droping null values

In addition to the masking used before, there are the convenience methods, `dropna()` (which removes NA values) and `fillna()` (which fills in NA values). For a `Series`, the result is straightforward:

In [39]:
data.dropna()

0        1
2    hello
dtype: object

In [40]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


We cannot drop single values from a `DataFrame`; we can only drop full rows or full columns. Depending on the application, you might want one or the other, so `dropna()` gives a number of options for a `DataFrame`.

By default, `dropna()` will drop all rows in which any null value is present:


In [41]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternatively, you can drop NA values along a different axis; `axis=1` drops all columns containing a null value:

In [46]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


But this drops some good data as well; you might rather be interested in dropping rows or columns with all NA values, or a majority of NA values. This can be specified through the `how` or `thresh` parameters, which allow fine control of the number of nulls to allow through.

The default is `how='any'`, such that any row or column (depending on the axis keyword) containing a null value will be dropped. You can also specify `how='all'`, which will only drop rows/columns that are all null values:


In [47]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [48]:
# This will drop columns with all NaN values
df.dropna(axis='columns', how="all")

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


For finer-grained control, the `thresh` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [52]:
# Atleast 3 NaN can be kept in a row or column
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


## Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a vaid value. This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values. You could do this in-place using the `isnull()` method as a mask, but because it is such a common operation Pandas provides the `fillna()` method, which returns a copy of the array with the null values replaced.

In [53]:
data = pd.Series([1, np.nan, 2, None, 3], 
                 index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [57]:
# Fill NA with zero
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [58]:
# forward-fill to propagte the previous
# value forward
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [61]:
# back-fill to propagate the next values backward
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For `DataFrame`s, the options are similar, but we can also be specify an `axis` along which the fill take place:

In [62]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [65]:
df.fillna(method='ffill', axis='columns')

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [66]:
df.fillna(method='ffill', axis='rows')

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


Notice that if a previous values is not available during a forward fill, the NA value remains.